In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
from variations import *
from transforms import *
from flame_io import *
from animation import *
from chaos_game import *
from music import *

---------------------------------

In [ ]:
# Run this to see how the variations behave
for vari in all_variations:
    vari().visualize()

In [ ]:
# Sierpinski's Gasket (https://en.wikipedia.org/wiki/Sierpinski_triangle)
# This is a common fractal that can be rendered, among other ways, using the chaos game
# I put it here to demonstrate how to do something obvious and known, and also to test the renderer
# NOTE: I don't think the Mandelbrot set can be rendered this way :'(

@variation
def gasket0(x, y, *crap):
    return x/2, y/2

@variation
def gasket1(x, y, *crap):
    return (x+1)/2, y/2

@variation
def gasket2(x, y, *crap):
    return x/2, (y+1)/2
gasket0 = gasket0(1, 0, 0, 0, 1, 0)
gasket1 = gasket1(1, 0, 0, 0, 1, 0)
gasket2 = gasket2(1, 0, 0, 0, 1, 0)

def sierpinski_gasket(**display_args):
    t = [gasket0, gasket1, gasket2]
    c, img, _ = chaos_game(t, **display_args)
    display_flame(img, renderer=None)
    save_flame(img, 'sierpinski_gasket', renderer=None, transes=t)

sierpinski_gasket(iters=30, n=100000, xmin=0, xmax=1, ymin=0, ymax=1, xres=512, yres=512)

In [ ]:
# FINALLY, we can render some fractal flame art
# This cell sets up a new flame configuration and renders it

flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 50,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': 100,
    'xres': 1440,
    'yres': 1080,
    'supersample': 1,
    'vibrancy': 1,
    'saturation': None,
}
transes = random_transform()
cond, img, _ = chaos_game(transes, **flame_settings)
display_flame(img, renderer=None, **flame_settings)

In [ ]:
print(cond.max())

In [ ]:
# This cell demonstrates how to pick up where you left off and continue rendering 
# (creates new batch of points, but that doesn't really matter)
# (Make sure not to change any render parameters, only display parameters or run configurations)
flame_settings['resume'] = cond
flame_settings['n'] = 1000000
flame_settings['iters'] = 200

cond = chaos_game(transes, **flame_settings)

In [ ]:
# This cell is for playing with display parameters without changing the render
from time import time
s1 = time()
display_flame(cond, renderer=theano_renderer, **flame_settings)
print(time() - s1)
s1 = time()
display_flame(cond, **flame_settings)
print(time() - s1)

In [ ]:
import theano
theano.config.device

In [ ]:
save_flame(cond, 'flame101', transes=transes, **flame_settings)

In [ ]:
# Save the flame to an image file (saves both to png and jpg)
flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 500,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': 100,
    'xres': 1440*2,
    'yres': 1080*2,
    'supersample': 2,
    'vibrancy': 1,
    'saturation': 3,
    'renderer': theano_renderer,
}

for i in range(52, 100):
    transes = random_transform()
    cond = chaos_game(transes, **flame_settings)
    save_flame(cond, 'flame%d' % i, transes=transes, vibrancy=1.0, saturation=3.0)

In [ ]:
'''
TODO:
Performance optimization (& parallelization)
Implement density estimation
Animate
'''

In [ ]:
flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 50,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': False,
    'xres': 1440,
    'yres': 1080,
    'supersample': 1,
    'vibrancy': 1,
    'saturation': None,
}
theano.config.optimizer = 'fast_run'

f_resume = None
odd_transform = random_transform()
for t in odd_transform:
    t.variations = [AnimatedVariation(var) for var in t.variations]
    for var in t.variations:
        var.step()
for i in range(600):
    cond, img, f_resume = chaos_game(odd_transform, reuse_func=f_resume, **flame_settings)
    display_flame(img, renderer=None, **flame_settings)
    save_flame(img, 'animated_flame_%d' % i, renderer=None, **flame_settings)
    for trans in odd_transform:
        for var in trans.variations:
            var.step()
    del cond

In [ ]:
for music_anim in range(4, 11):
    flame_settings = {
        'n': 1000000,
        'n_runs': 1,
        'iters': 100,
        'xmin': -1.166,
        'xmax': 1.166,
        'ymin': -1,
        'ymax': 1,
        'and_display': False,
        'xres': 1440,
        'yres': 1080,
        'supersample': 2,
        'vibrancy': 1,
        'saturation': None,
    }

    f_resume = None
    odd_transform = random_transform()
    music = Music(framerate=15)
    #music.visualize()

    for t in odd_transform:
        t.variations = [MusicalVariation(music, var) for var in t.variations]
        for var in t.variations:
            var.step()
    for i in range(len(music)):
        cond, img, f_resume = chaos_game(odd_transform, reuse_func=f_resume, **flame_settings)
        #display_flame(img, renderer=None, **flame_settings)
        save_flame(img, 'music_animation%d/animated_flame_%d' % (music_anim, i), renderer=None, **flame_settings)
        for trans in odd_transform:
            for var in trans.variations:
                var.step()
        del cond

In [ ]:
flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 50,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': False,
    'xres': 1440,
    'yres': 1080,
    'supersample': 2,
    'vibrancy': 1,
    'saturation': None,
}
theano.config.optimizer = 'fast_run'

f_resume = None
odd_transform = random_transform()
music = Music(framerate=15)
#music.visualize()

for t in odd_transform:
    t.variations = [MusicalVariation(music, var) for var in t.variations]
    for var in t.variations:
        var.step()
for i in range(len(music)):
    cond, img, f_resume = chaos_game(odd_transform, reuse_func=f_resume, **flame_settings)
    display_flame(img, renderer=None, **flame_settings)
    save_flame(img, 'animated_flame_%d' % i, renderer=None, **flame_settings)
    for trans in odd_transform:
        for var in trans.variations:
            var.step()
    del cond

In [ ]:
def pixel_kde_smoothing(img):
    yres, xres, num_chans = img.shape
    out_image = np.zeros_like(img)
    xidxs = np.arange(-xres, xres+1).reshape((1, 2*xres + 1))
    yidxs = np.arange(-yres, yres+1).reshape((2*yres + 1, 1))
    
    log_alpha = np.log(img[:,:,3])
    mags = np.cast['int'](np.ceil(log_alpha))
    max_mag = np.max(mags)
    
    full_gaussian = {sigma: 
        np.exp(-((xidxs - xres)**2 / (2 * sigma**2) + (yidxs - yres)**2 / (2 * sigma**2)))
        for sigma in range(1, max_mag + 1)
    }
    
    def gaussian(x, y, mag):
        return full_gaussian[mag][yres-y:2*yres-y, xres-x:2*xres-x]
    
    print(gaussian(0, 0, 1).shape)
        
    for y in tqdm(range(yres)):
        for x in range(xres):
            alpha = mags[y, x]
            if alpha > 1:
                g = gaussian(x, y, alpha)
                out_image += alpha * np.expand_dims(g, 2) * img[y, x, :].reshape((1, 1, num_chans))
    return out_image
        
print(img.shape)
new_img = pixel_kde_smoothing(img.copy())
display_flame(new_img, renderer=None, **flame_settings)

In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import theano.misc.pycuda_init
import theano.sandbox.cuda as cuda
import theano.tensor as T

class PyCUDAKDE2DImg(theano.Op):
    __props__ = ()
    
    def __init__(self, smoothness=1, window_size=7, **condense_args):
        self.smoothness = smoothness
        self.window_size = window_size
        get = make_get(condense_args)
        self.xres = get('xres') * get('supersample')
        self.yres = get('yres') * get('supersample')
        self.num_chans = get('num_chans')
        print(self.xres)
    
    def make_node(self, inp):
        #max_alpha = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(T.max(inp[:,:,3])))
        inp = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(inp))
        out_t = cuda.CudaNdarrayType((False, False, False), dtype='float32')()
        assert inp.dtype == 'float32'
        return theano.Apply(self, [inp], [out_t])
        
    def make_thunk(self, node, storage_map, _, _2):

        # Use a std dev of 1/(alpha/max_alpha/smoothness)
        # 99.7% of the information is in the first three std dev's, so we'll stop there
        code = '''
        __device__ bool valid(int x, int y) {{
            return x >= 0 && x < {xres} && y >= 0 && y < {yres};
        }}
        
        __device__ void add(float *out, int x, int y, float r, float g, float b, float a, float gauss) {{
            atomicAdd(&out[(y * {xres} + x) * {num_chans} + 0], r * gauss);
            atomicAdd(&out[(y * {xres} + x) * {num_chans} + 1], g * gauss);
            atomicAdd(&out[(y * {xres} + x) * {num_chans} + 2], b * gauss);
            atomicAdd(&out[(y * {xres} + x) * {num_chans} + 3], a * gauss);
        }}
        
        __global__ void kde2d(const float *input, const float max_alpha, const unsigned int offset_x, const unsigned int offset_y, float *out) {{
            register int _x = blockIdx.x + offset_x;
            register int _y = blockIdx.y + offset_y;
            
            register int dx = threadIdx.x;
            register int dy = threadIdx.y;
            
            register float r = input[(_y * {xres} + _x) * {num_chans} + 0];
            register float g = input[(_y * {xres} + _x) * {num_chans} + 1];
            register float b = input[(_y * {xres} + _x) * {num_chans} + 2];
            register float a = input[(_y * {xres} + _x) * {num_chans} + 3];
            
            if(a < __logf(max_alpha))
                return;
            
            float sigma = 0.3f * a / max_alpha * (float){smoothness};
            
            float gaussian = __expf((dx)*(dx)/(float)(2*sigma*sigma) + (dy)*(dy)/(float)(2*sigma*sigma));
            
            int xp = _x + dx;
            int yp = _y + dy;
            int xm = _x - dx;
            int ym = _y - dy;
            
            if(valid(xp, yp))
                add(out, xp, yp, r, g, b, a, gaussian);
            if(dx != 0 || dy != 0) {{
                if(valid(xm, yp))
                    add(out, xm, yp, r, g, b, a, gaussian);
                if(valid(xp, ym))
                    add(out, xp, ym, r, g, b, a, gaussian);
                if(valid(xm, ym))
                    add(out, xm, ym, r, g, b, a, gaussian);
            }}
            out[(_y * {xres} + _x) * {num_chans}] = sigma;
        }}
        '''.format(**{
                'window_size': self.window_size,
                'smoothness': self.smoothness,
                'xres': self.xres,
                'yres': self.yres,
                'num_chans': self.num_chans,
            })
        #print(code)
        #comp = pycuda.compiler.compile(code)
        #print(comp)
        #mod = pycuda.driver.module_from_buffer(comp)
        mod = SourceModule(code)
        cuda_func = mod.get_function('kde2d')
        #cuda_func.prepare('PFIIP')
        
        inputs = [storage_map[v] for v in node.inputs]
        outputs = [storage_map[v] for v in node.outputs]
        
        (_inp,) = inputs
        (_out,) = outputs

        def run_kde():
            stream = drv.Stream()
            
            if _out[0] is None or _out[0].shape != _inp[0].shape:
                _out[0] = cuda.CudaNdarray.zeros(_inp[0].shape)
            
            block_size = (self.window_size+1, self.window_size+1, 1)
            stepsize = 32
            height, width, _ = _inp[0].shape
            __max_alpha = np.asarray(_inp[0])[:,:,3].max()
            print(__max_alpha)
            for offset_x in range(0, stepsize*20+1, stepsize):
                for offset_y in range(0, stepsize*20+1, stepsize):
                    cuda_func(_inp[0], __max_alpha, np.uint32(offset_x), np.uint32(offset_y), _out[0], block=block_size, grid=(stepsize, stepsize))
            #        grid_size = (min(stepsize, width-offset_x), min(stepsize, height-offset_y))
            #        cuda_func.prepared_async_call(grid_size, block_size, stream, drv.In(_inp[0]), _max_alpha[0], np.uint32(offset_x), np.uint32(offset_y), drv.Out(_out[0]))
            #stream.synchronize()
            #cuda_func(_inp[0], _max_alpha[0], _out[0], block=block_size, grid=grid_size)
            
        return run_kde
    
from time import time
tmp_img = img.copy()
t_img = T.ftensor3()
kde_func = PyCUDAKDE2DImg(smoothness=1, window_size=7, **flame_settings)
#kde_func.make_thunk(None, None, None, None)
filtered = kde_func(t_img)
runner = theano.function([t_img], filtered, allow_input_downcast=True)
s1 = time()
new_img = np.asarray(runner(tmp_img))
s2 = time()
print(s2 - s1)
print(new_img)
print(np.where(np.isnan(new_img)))
print(np.where(np.isinf(new_img)))
print(np.where(np.isneginf(new_img)))

display_flame(tmp_img, renderer=None, **flame_settings)
display_flame(new_img, renderer=None, **flame_settings)

In [ ]:
'''TODO:
Make music visualization work better...
    - Try finding the peak frequency in each range (argmax), and use that as a possible value
'''